Cellule 1 : Importer les bibliothèques

In [11]:
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

Cellule 2 : Lecture du contenu HTML

In [7]:
# Charger le fichier HTML local
with open("oddsportal.html", "r", encoding="utf-8") as file:
    html = file.read()

Cellule 3 : Analyser le contenu avec BeautifulSoup

In [9]:
# Analyse avec BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

# Exemple : Recherche d'un élément spécifique
tabs = soup.select_one("div.tabs")
if tabs:
    print("Élément 'div.tabs' trouvé.")
else:
    print("Aucun élément 'div.tabs' trouvé.")

Élément 'div.tabs' trouvé.


Cellule 4 : Extraction des données

In [10]:
# Récupération des éléments visibles et cachés
try:
    valuebets = tabs.select("div.visible") + tabs.select("div.hidden")
except Exception as e:
    raise Exception(f"Erreur lors de la récupération des éléments visibles et cachés : {e}")

# Initialisation du dictionnaire pour stocker les données
data = {
    'sports': [], 'countries': [], 'leagues': [], 'pronos': [],
    'date': [], 'time': [], 'team_1': [], 'team_2': [],
    'outcome': [], 'bookmaker': [], 'odds': [], 'value': [], 
    'probability': []
}

# Extraction des données des paris
for valuebet in valuebets:
    try:
        header = valuebet.select("a")
        data['sports'].append(header[0].text.strip() if len(header) > 0 else None)
        data['countries'].append(header[1].text.strip() if len(header) > 1 else None)
        data['leagues'].append(' '.join(header[2].text.split()) if len(header) > 2 else None)
    except Exception as e:
        print(f"Erreur lors de l'extraction des données de l'en-tête : {e}")

# Extraction des informations sur les matchs
match_info = tabs.find_all("div", class_="flex min-h-[90px] w-full")
for match in match_info:
    try:
        p_elements = match.select("p")
        match_data = [p.text.strip() for p in p_elements]
        if len(match_data) >= 9:  # Vérification pour éviter les erreurs d'indice
            data['pronos'].append(match_data[0])
            data['date'].append(match_data[1])
            data['time'].append(match_data[2])
            data['team_1'].append(match_data[3])
            data['team_2'].append(match_data[4])
            data['outcome'].append(match_data[5])
            data['odds'].append(match_data[6])
            data['value'].append(match_data[7])
            data['probability'].append(match_data[8])
    except Exception as e:
        print(f"Erreur lors de l'extraction des informations sur les matchs : {e}")

# Extraction des informations sur les bookmakers
bookmaker_info = tabs.find_all("div", class_="h-[25px] w-[75px]")
for bookmaker in bookmaker_info:
    try:
        img = bookmaker.find("img")
        data['bookmaker'].append(img['alt'] if img and 'alt' in img.attrs else None)
    except Exception as e:
        print(f"Erreur lors de l'extraction des informations sur les bookmakers : {e}")

Cellule 7 : Analyse des données avec Pandas

In [12]:
# Création d'un DataFrame pandas à partir des données collectées
try:
    df = pd.DataFrame(data)
except Exception as e:
    raise Exception(f"Erreur lors de la création du DataFrame : {e}")


# Nettoyage de la colonne "probability" pour enlever le caractère '%'
df['probability'] = pd.to_numeric(df['probability'].apply(lambda p: p.replace('%', '') if isinstance(p, str) else None), errors='coerce')

# Remplacement de "Tomorr." par la date de demain et "Today." par la date d'aujourd'hui dans la colonne "date"
today = datetime.now()
df['date'] = df['date'].apply(lambda date: date.replace(',', '').replace("Tomorr.", (today + timedelta(days=1)).strftime('%d %b')).replace("Today.", today.strftime('%d %b')) if isinstance(date, str) else None)

# Ajout de l'année actuelle aux dates
current_year = today.year
df['date'] = df['date'].apply(lambda date: f"{date} {current_year}" if isinstance(date, str) else None)

# Uniformisation du format des colonnes "date" et "time"
df['date'] = pd.to_datetime(df['date'], format='%d %b %Y', errors='coerce')
df['time'] = pd.to_datetime(df['time'], format='%H:%M', errors='coerce').dt.strftime('%H:%M')

# Conversion des colonnes "value" et "odds" en float
df['value'] = pd.to_numeric(df['value'], errors='coerce')
df['odds'] = pd.to_numeric(df['odds'], errors='coerce')

# Trier le DataFrame par valeur décroissante de la colonne "probability"
df.sort_values(by='probability', ascending=False, inplace=True)

# Sauvegarde du DataFrame dans un fichier CSV
try:
    df.to_csv('oddsportal_data.csv', index=False)
    print("Les données ont été sauvegardées avec succès dans 'oddsportal_data.csv'.")
except Exception as e:
    raise Exception(f"Erreur lors de la sauvegarde des données dans un fichier CSV : {e}")

# Affichage des 5 premières lignes du DataFrame
df.head()

Les données ont été sauvegardées avec succès dans 'oddsportal_data.csv'.


,sports,countries,leagues,pronos,date,time,team_1,team_2,outcome,bookmaker,odds,value,probability
31,Hockey,USA,NHL,"Home/Away, FT including OT",2025-04-27,20:00,St. Louis Blues,Winnipeg Jets,1,Betfury,285,1.88,46.37
38,Futsal,Poland,Ekstraklasa,1X2,2025-04-26,20:00,Ruda Slaska,Bielsko-Biala,X,1xBet,1700,1.70,8.95
35,Handball,France,Starligue,DNB,2025-04-26,21:00,PSG,Tremblay,2,bet365,2000,1.80,8.78
7,Football,Egypt,Premier League,"O/U 3.5, 1st Half",2025-05-01,17:00,Ceramica Cleopatra,National Bank Egypt,Over,1xBet,3300,2.40,8.36
25,Football,Spain,Primera RFEF - Group 2,"O/U 3.5, 1st Half",2025-04-27,20:30,Hercules,Alcorcon,Over,1xBet,2500,2.08,8.23
